In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sys
import os
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))
from lib.preprocessing_data import Preprocessing
from lib.data_preparation import DataPreparation
from lib.word2vec import Word2Vec

In [8]:
def train(preprocessing_method, vectorizer, model, description):
    data_set = DataPreparation().load_data()
    data_set = preprocessing_method(data_set)
    data_set = data_set.sample(frac=1, random_state=42).reset_index(drop=True)

    split_index = int(len(data_set) * 0.8)
    train_df = data_set[:split_index]
    test_df = data_set[split_index:]

    X_train = train_df['processed_tweet']
    y_train = train_df['label'].to_numpy().astype(int)
    X_test = test_df['processed_tweet']
    y_test = test_df['label'].to_numpy().astype(int)
    
    
    
    if description.split()[0] == 'word2vec':
        # train_word2vec = Word2Vec(data_set, data_set.shape[0]).word2vec
        # X_train_vector = train_word2vec(X_train)
        # X_test_vector = train_word2vec(X_test)
        print(X_test.shape)
    else:
        X_train_vector = vectorizer.fit_transform(X_train.apply(lambda row: ' '.join(row)))
        X_test_vector = vectorizer.transform(X_test.apply(lambda row: ' '.join(row)))
    
    model.fit(X_train_vector, y_train)
    y_pred = model.predict(X_test_vector)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {description}: {accuracy}")
    return description, accuracy

In [1]:
preprocessing_methods = {
    # "lemmatization": Preprocessing().lemmatization,
    "tokenization": Preprocessing().tokenization,
    # "stemming": Preprocessing().stemming,
}
vectorizers = {
    # "TF-IDF": TfidfVectorizer(),
    # "Bag of words": CountVectorizer(binary=False),
    # "o or 1 if word exists": CountVectorizer(binary=True),
    "word2vec": PCA(n_components=2)
}
models = {
    "Logistic Regression": LogisticRegression(),
    "MultinomialNB": MultinomialNB(),
    "Linear SVC": LinearSVC()
}

# Dynamically run all combinations
results = []

for preprocessing_name, preprocessing_method in preprocessing_methods.items():
    for vectorizer_name, vectorizer in vectorizers.items():
        for model_name, model in models.items():
            description = f"{vectorizer_name} + {preprocessing_name} + {model_name}"
            desc, accuracy = train(preprocessing_method, vectorizer, model, description)
            results.append((desc, accuracy))


In [4]:
import matplotlib.pyplot as plt

results = []

for preprocessing_name, preprocessing_method in preprocessing_methods.items():
    for vectorizer_name, vectorizer in vectorizers.items():
        for model_name, model in models.items():
            description = f"{vectorizer_name} + {preprocessing_name} + {model_name}"
            desc, accuracy = train(preprocessing_method, vectorizer, model, description)
            results.append((desc, accuracy))

# results_df = pd.DataFrame(results, columns=["Description", "Accuracy"])

# plt.figure(figsize=(12, 8))
# results_df.sort_values(by="Accuracy", ascending=False, inplace=True)
# plt.barh(results_df["Description"], results_df["Accuracy"], color="skyblue")
# plt.xlabel("Accuracy")
# plt.title("Accuracy of Different Preprocessing, Vectorizer, and Model Combinations")
# plt.tight_layout()
# plt.show()

In [5]:
results_df = pd.DataFrame(results, columns=["Description", "Accuracy"])

results_df["Model"] = results_df["Description"].apply(lambda x: x.split("+")[-1].strip())

model_colors = {
    "Logistic Regression": "red",
    "MultinomialNB": "green",
    "Linear SVC": "blue",
}

results_df["Color"] = results_df["Model"].map(model_colors)

results_df.sort_values(by="Accuracy", ascending=False, inplace=True)

plt.figure(figsize=(12, 8))
plt.barh(
    results_df["Description"], 
    results_df["Accuracy"], 
    color=results_df["Color"]
)

handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in model_colors.values()]
labels = model_colors.keys()
plt.legend(handles, labels, title="Models")

plt.xlabel("Accuracy")
plt.title("Accuracy of Different Preprocessing, Vectorizer, and Model Combinations")
plt.tight_layout()
plt.show()